# **Google’s STAC**

The Google Brain team has introduced STAC, a semi-supervised learning (SSL) framework to perform object detection in a simplified way. STAC outperforms most supervised approaches with greater data efficiency and simplicity. Further, STAC opens a new gateway to SSL-based visual object detection.

To read about it more, please refer [this](https://analyticsindiamag.com/googles-stac-ssl-framework-for-object-detection/) article.

## **Python Implementation of STAC**

STAC requires a Python environment with TensorFlow version 1.14 and a CUDA runtime with 8 GPUs. Most of this implementation references the official source code repository of STAC. Clone the source code to the local (or virtual) environment using the following command.

In [1]:
!git clone https://github.com/google-research/ssl_detection.git

Cloning into 'ssl_detection'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 329 (delta 28), reused 320 (delta 23), pack-reused 0
Receiving objects: 100% (329/329), 1.63 MiB | 16.85 MiB/s, done.
Resolving deltas: 100% (28/28), done.


Create the environment by installing the dependencies using the following commands.

In [2]:
%%bash
cd /content/ssl_detection/
sudo apt install python3-dev python3-virtualenv python3-tk imagemagick
virtualenv -p python3 --system-site-packages env3
. env3/bin/activate
pip install -r requirements.txt
python -c 'import tensorflow as tf; print(tf.__version__)'
# install coco apis
pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' 

Reading package lists...
Building dependency tree...
Reading state information...
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
python3-tk is already the newest version (3.6.9-1~18.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
  python-pip-whl python3-pkg-resources
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  f



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 26.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
bash: line 3: virtualenv: command not found
bash: line 4: env3/bin/activate: No such file or directory
ERROR: xarray 0.18.2 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow

Prepare the COCO dataset using the following commands. Unzip the compressed files once the download is finished.

In [3]:
%%bash
mkdir -p /content/coco/
cd /content/coco/
wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
wget http://images.cocodataset.org/zips/train2017.zip
wget http://images.cocodataset.org/zips/val2017.zip
wget http://images.cocodataset.org/zips/unlabeled2017.zip
unzip annotations_trainval2017.zip -d .
unzip -q train2017.zip -d .
unzip -q val2017.zip -d .
unzip -q unlabeled2017.zip -d . 

Archive:  annotations_trainval2017.zip
  inflating: ./annotations/instances_train2017.json  
  inflating: ./annotations/instances_val2017.json  
  inflating: ./annotations/captions_train2017.json  
  inflating: ./annotations/captions_val2017.json  
  inflating: ./annotations/person_keypoints_train2017.json  
  inflating: ./annotations/person_keypoints_val2017.json  


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Similarly, download the VOC dataset and untar them using the following commands.

In [4]:
%%bash
mkdir -p /content/voc/
cd /content/voc/
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
tar -xf VOCtrainval_06-Nov-2007.tar
tar -xf VOCtest_06-Nov-2007.tar
tar -xf VOCtrainval_11-May-2012.tar 

mkdir: cannot create directory ‘/content/voc/’: No space left on device
bash: line 2: cd: /content/voc/: No such file or directory
--2021-06-03 08:59:07--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

     0K                                                         0% 0.00 =0s


Cannot write to ‘VOCtrainval_06-Nov-2007.tar’ (No space left on device).
--2021-06-03 08:59:07--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (

Generate annotated, and unannotated data split from the downloaded annotated dataset.

In [5]:
%%bash
cd /content/ssl_detection/prepare_datasets
for seed in 1 2 3 4 5; do
  for percent in 1 2 5 10 20; do
    python3 prepare_coco_data.py --percent $percent --seed $seed &
  done
done 

Traceback (most recent call last):
  File "prepare_coco_data.py", line 27, in <module>
    assert 'COCODIR' in os.environ, 'Set COCODIR in bash'
AssertionError: Set COCODIR in bash
Traceback (most recent call last):
  File "prepare_coco_data.py", line 27, in <module>
    assert 'COCODIR' in os.environ, 'Set COCODIR in bash'
AssertionError: Set COCODIR in bash
Traceback (most recent call last):
  File "prepare_coco_data.py", line 27, in <module>
    assert 'COCODIR' in os.environ, 'Set COCODIR in bash'
AssertionError: Set COCODIR in bash
Traceback (most recent call last):
  File "prepare_coco_data.py", line 27, in <module>
    assert 'COCODIR' in os.environ, 'Set COCODIR in bash'
AssertionError: Set COCODIR in bash
Traceback (most recent call last):
  File "prepare_coco_data.py", line 27, in <module>
    assert 'COCODIR' in os.environ, 'Set COCODIR in bash'
AssertionError: Set COCODIR in bash
Traceback (most recent call last):
  File "prepare_coco_data.py", line 27, in <module>
    asse

Download JSON files for the downloaded VOC dataset and COCO dataset.

In [6]:
%%bash
cd /content/
wget https://storage.cloud.google.com/gresearch/ssl_detection/STAC_JSON.tar
tar -xf STAC_JSON.tar.gz 

--2021-06-03 08:59:12--  https://storage.cloud.google.com/gresearch/ssl_detection/STAC_JSON.tar
Resolving storage.cloud.google.com (storage.cloud.google.com)... 173.194.216.138, 173.194.216.113, 173.194.216.100, ...
Connecting to storage.cloud.google.com (storage.cloud.google.com)|173.194.216.138|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=cds&passive=1209600&continue=https://storage.cloud.google.com/gresearch/ssl_detection/STAC_JSON.tar&followup=https://storage.cloud.google.com/gresearch/ssl_detection/STAC_JSON.tar [following]
--2021-06-03 08:59:13--  https://accounts.google.com/ServiceLogin?service=cds&passive=1209600&continue=https://storage.cloud.google.com/gresearch/ssl_detection/STAC_JSON.tar&followup=https://storage.cloud.google.com/gresearch/ssl_detection/STAC_JSON.tar
Resolving accounts.google.com (accounts.google.com)... 173.194.218.84, 2607:f8b0:400c:c15::54
Connecting to accounts.google.com 

Download an untrained FasterRCNN backbone model 

In [7]:
%%bash
cd /content/coco/
wget http://models.tensorpack.com/FasterRCNN/ImageNet-R50-AlignPadding.npz 

--2021-06-03 08:59:18--  http://models.tensorpack.com/FasterRCNN/ImageNet-R50-AlignPadding.npz
Resolving models.tensorpack.com (models.tensorpack.com)... 185.207.105.29
Connecting to models.tensorpack.com (models.tensorpack.com)|185.207.105.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95179737 (91M) [application/octet-stream]
Saving to: ‘ImageNet-R50-AlignPadding.npz’

     0K                                                         0% 0.00 =0s


Cannot write to ‘ImageNet-R50-AlignPadding.npz’ (No space left on device).


Prepare COCO dataset for training. Generate a path to save the model checkpoints during training. Prepare CUDA for training the FasterRCNN backbone model with 8 GPUs using the following commands. (Users can opt for 4, 16, or 32 GPUs)

In [8]:
%%bash
cd /content/ssl_detection/detection
# Labeled and Unlabeled datasets
DATASET=coco_train2017.1@10
UNLABELED_DATASET=${DATASET}-unlabeled
# PATH to save trained models
CKPT_PATH=result/${DATASET}
# PATH to save pseudo labels for unlabeled data
PSEUDO_PATH=${CKPT_PATH}/PSEUDO_DATA
# Train with 8 GPUs
export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 

Train the teacher model- FasterRCNN with the above settings on the prepared dataset.

In [14]:
%%bash
python train_stg1.py \
    --logdir ${CKPT_PATH} --simple_path --config \
    BACKBONE.WEIGHTS=/content/coco/ImageNet-R50-AlignPadding.npz \
    DATA.BASEDIR=/content/coco/ \
    DATA.TRAIN="('${DATASET}',)" \
    MODE_MASK=False \
    FRCNN.BATCH_PER_IM=64 \
    PREPROC.TRAIN_SHORT_EDGE_SIZE="[500,800]" \
    TRAIN.EVAL_PERIOD=20 \
    TRAIN.AUGTYPE_LAB='default' 

python3: can't open file 'train_stg1.py': [Errno 2] No such file or directory


Evaluate the teacher model using the following commands.

In [10]:
%%bash
if [ ! -d ${PSEUDO_PATH} ]; then
    mkdir -p ${PSEUDO_PATH}
fi
# model-180000 is the last checkpoint
# save eval.json at $PSEUDO_PATH

python3 predict.py \
    --evaluate ${PSEUDO_PATH}/eval.json \
    --load "${CKPT_PATH}"/model-180000 \
    --config \
    DATA.BASEDIR=/content/coco/ \
    DATA.TRAIN="('${UNLABELED_DATASET}',)" 

python3: can't open file 'predict.py': [Errno 2] No such file or directory


Once the teacher model is trained, it can be used to prepare pseudo labels for the unannotated images.

In [11]:
%%bash
python3 predict.py \
    --predict_unlabeled ${PSEUDO_PATH} \
    --load "${CKPT_PATH}"/model-180000 \
    --config \
    DATA.BASEDIR=/content/coco/ \
    DATA.TRAIN="('${UNLABELED_DATASET}',)" \
    EVAL.PSEUDO_INFERENCE=True 

python3: can't open file 'predict.py': [Errno 2] No such file or directory


Train the STAC detector with necessary configurations using the following command. It should be noted that training may take hours based on memory availability.

In [12]:
%%bash
python3 train_stg2.py \
    --logdir=${CKPT_PATH}/STAC --simple_path \
    --pseudo_path=${PSEUDO_PATH} \
    --config \
    BACKBONE.WEIGHTS=/content/coco/ImageNet-R50-AlignPadding.npz \
    DATA.BASEDIR=/content/coco/ \
    DATA.TRAIN="('${DATASET}',)" \
    DATA.UNLABEL="('${UNLABELED_DATASET}',)" \
    MODE_MASK=False \
    FRCNN.BATCH_PER_IM=64 \
    PREPROC.TRAIN_SHORT_EDGE_SIZE="[500,800]" \
    TRAIN.EVAL_PERIOD=20 \
    TRAIN.AUGTYPE_LAB='default' \
    TRAIN.AUGTYPE='strong' \
    TRAIN.CONFIDENCE=0.9 \
    TRAIN.WU=2 

python3: can't open file 'train_stg2.py': [Errno 2] No such file or directory


A trained model can be directly deployed for inference.

#**Related Articles:**

> * [Google STAC](https://analyticsindiamag.com/googles-stac-ssl-framework-for-object-detection/)

> * [Point Transformers](https://analyticsindiamag.com/how-point-transformer-excels-in-3d-image-processing/)

> * [Comparison of Transfer Learning with Multi Class Classification](https://analyticsindiamag.com/practical-comparison-of-transfer-learning-models-in-multi-class-image-classification/)

> * [Fruit Recognition with CNN](https://analyticsindiamag.com/fruit-recognition-using-the-convolutional-neural-network/)

> * [Semantic Segmentation Using TensorFlow Keras](https://analyticsindiamag.com/semantic-segmentation-using-tensorflow-keras/)

> * [Convert Image to Pencil Sketch](https://analyticsindiamag.com/converting-image-into-a-pencil-sketch-in-python/)

> * [Image Classification Task with and without Data Augmentation](https://analyticsindiamag.com/image-data-augmentation-impacts-performance-of-image-classification-with-codes/)
